## **Notebook 1: Introduction to CuPy**

**Introduction to Workshop Lab Environment**

Markdown cells contain plain text, and code cells contain interactive Python code.

In [ ]:
print("Hello World!!!")

We can also run shell commands by prepending an exclamation mark to our code cells. Let's query for some basic information about our system. `nvidia-smi` is like `top` for NVIDIA GPUs.

In [ ]:
!nvidia-smi

Let's check out the connection topology of our system.

In [ ]:
!nvidia-smi topo -m

And finally, let's check out the type of CPU we were allocated.

In [ ]:
!lscpu

---

**Introduction to CuPy**

NumPy is a widely used library for numerical computing in Python. 

In [ ]:
import numpy as np

size = 512

A = np.random.randn(size, size)

%timeit -n 5 Q, R = np.linalg.qr(A)

CuPy uses a NumPy-like interface. Porting a Numpy code to CuPy can be as simple as changing your import statement. In this workshop, we'll always use `import cupy as cp` for clarity.

In [ ]:
import cupy as cp

size = 512

A = cp.random.randn(size, size)

Q, R = cp.linalg.qr(A)
%timeit -n 5 Q, R = cp.linalg.qr(A) ; cp.cuda.Device().synchronize()

We already see a substantial speedup with no real code changes! 

Notice the additional call to `cp.cuda.Device().synchronize()` in the CuPy version. GPU kernel calls are asynchronous with respect to the CPU. Our call to `synchronize()` ensures the GPU finishes to completion, so we can accurately measure  the elapsed time. We don't generally need to add these calls to production CuPy codes.

NumPy is typically used to perform computations on _arrays_ of data. The data is stored in the `numpy.ndarray` object. CuPy implements a similar class called the `cupy.ndarray`. But while the `numpy.ndarray` data resides in host memory, the contents of a `cupy.ndarray` persistent in GPU memory. CuPy provides several helper functions to convert between Cupy and NumPy `ndarrays` - facilitating data transfer to/from the GPU device.

In [ ]:
#Initialize the data on the host
A_cpu = np.array([[1, 2, 3], [4, 5, 6]], np.int32)

print("A_cpu is a", type(A_cpu))
print("With initial values:\n", A_cpu)

#Copy data, host to device
A_gpu = cp.asarray(A_cpu)
print("A_gpu is a", type(A_gpu))

#Square the data on the device
A_gpu = cp.square(A_gpu)

#Copy data, device to host
A_cpu = cp.asnumpy(A_gpu)

print("Squared values:\n", A_cpu)


Note that NumPy and CuPy ndarrys are not implicitly convertible.

In [ ]:
cp.square(A_cpu)

CuPy is useful for programming multi-GPU nodes as well. We can orchestrate computation, data movement, and other low-level CUDA operations with functions in the `cupy.cuda` namespace.

In [ ]:
#Initialize array on GPU 1
with cp.cuda.Device(1):
    A_gpu_1 = cp.array([[1, 2, 3], [4, 5, 6]], cp.int32)

#Copy array from GPU 1 to GPU 0
A_gpu_0 = cp.asarray(A_gpu_1)

print(A_gpu_0)


The GPU is a powerhouse of parallel computing performance, and can process math operations much more quickly than the CPU. This is easy to see by comparing performance of CuPy vs NumPy, particularly for dense linear algebra operations. Let's look at a multiplication of 4096x4096 matrices. Notice the similarity of the two versions of the code (NumPy and CuPy).

In [ ]:
import math
from time import perf_counter

size = 4096

start_time = perf_counter( )
A_cpu = np.random.uniform(low=-1.0, high=1.0, size=(size,size) ).astype(np.float32)
B_cpu = np.random.uniform(low=-1., high=1., size=(size,size) ).astype(np.float32)
C_cpu = np.matmul(A_cpu,B_cpu)
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for numpy = %g seconds.' % (stop_time - start_time) )
print('')

del A_cpu
del B_cpu
del C_cpu



A_gpu = cp.random.uniform(low=-1.0, high=1.0, size=(size,size) ).astype(cp.float32)
B_gpu = cp.random.uniform(low=-1., high=1., size=(size,size) ).astype(cp.float32)
C_gpu = cp.matmul(A_gpu,B_gpu) #Exclude one-time JIT overhead
start_time = perf_counter( )
C_gpu = cp.matmul(A_gpu,B_gpu)
cp.cuda.Device(0).synchronize()
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for cupy = %g seconds.' % (stop_time - start_time) )
print('')

del A_gpu
del B_gpu
del C_gpu

The GPU's strenghts in computational throughput and memory bandwidth can lead to terrific application speedups. But we need to be considerate of two types of overhead when evaluating our problem for acceleration on the GPU with CuPy: kernel overhead, and data movement overhead.

---

**Kernel Overhead**

CuPy compiles kernel codes on-the-fly using JIT compilation. Therefore, there is a compilation overhead the first time a given function is called with CuPy. The compiled kernel code is cached, so compilation overhead is avoided for subsequent executions of the function.

In [ ]:
import time

size = 512
for _ in range(5):
    A = cp.random.randn(size, size).astype(np.float32)
    t1 = time.time()
    cp.linalg.det(A)
    cp.cuda.Device().synchronize()
    t2 = time.time()
    print('%.4f' % (t2 - t1))




You may also notice a one-time overhead upon first calling a CuPy function in a program. This overhead is associated with the creation of a CUDA context by the CUDA driver, which happens the first time any CUDA API is invoked in a program.

In addition, there is a CUDA kernel launch overhead that is penalized each time a GPU kernel is launched. The overhead is on the order of a few microseconds. For this reason, launching many small CUDA kernels in an application will generally lead to poor performance. The kernel launch overhead may dominate your runtime for very small problems, but for large datasets the overhead will be small compared to the actual GPU computation work.

In [ ]:
for size in [64, 128, 256, 512, 1024, 2048]:
    print("\nInput Matrix size: %d" % size, "x %d " % size)
    for xp in [np, cp]:
        A=xp.random.uniform(low=-1.0, high=1.0, size=(size,size) ).astype(xp.float32)
        xp.linalg.qr(A)#Exclude potential one-time JIT overhead
        t1 = time.time()
        xp.linalg.qr(A)
        cp.cuda.Device().synchronize()
        t2 = time.time()
        print(xp.__name__, '%f' % (t2 - t1))
        del A

It's clear that increasing the problem size can help amoritize the overhead of launching GPU kernels. Another common strategy is to merge multiple kernels together into a single combined kernel, reducing the total number of kernel launches in your program. CuPy supports kernel fusion in this manner via the `@cupy.fuse()` decorator.

In [ ]:
def squared_diff(x, y):
    return (x - y) * (x - y)

@cp.fuse
def fused_squared_diff(x, y):
    return (x - y) * (x - y)

size = 10000

x = cp.arange(size)
y = cp.arange(size)[::-1]

%timeit -n 10 squared_diff(x, y); cp.cuda.Device().synchronize()
%timeit -n 10 fused_squared_diff(x, y); cp.cuda.Device().synchronize()

del x
del y


---

**Data Movement Overhead**

Try to minimize data movement to or from the GPU. The FLOP rate and memory bandwidth of a GPU can process data much more quickly than it can be fed with data over the PCIe bus. This problem is being tackled with novel interconnect technologies like NVLink. But it's a real inbalance we have to deal with for now.
Let's look at an example where we initialize our input data GPU and then computes the dot product. Note that the result of the multiplication, the C matrix, is available on the GPU in case we need it later.

Notice again the similarity of the two parts of the code (NumPy and CuPy). They are virtually identical.

In [ ]:
size = int(1e8)

for i in range(3):
    print("Iteration ", i)
    start_time = perf_counter( )
    A_cpu=np.random.rand(size).astype(np.float32)
    B_cpu=np.random.rand(size).astype(np.float32)
    C_cpu = np.dot(A_cpu,B_cpu)
    stop_time = perf_counter( )
    cpu_time = stop_time - start_time
    print('numpy = %g seconds' % cpu_time )

    start_time = perf_counter( )
    A_gpu=cp.random.rand(size).astype(cp.float32)
    B_gpu=cp.random.rand(size).astype(cp.float32)
    C_gpu = cp.dot(A_gpu,B_gpu)
    cp.cuda.Device(0).synchronize()
    stop_time = perf_counter( )
    gpu_time = stop_time - start_time
    
    print('cupy = %g seconds' % gpu_time )
    print("Speedup = %.2f" % (cpu_time/gpu_time))
    print('')

But what if the input data for the `dot` operation resides in the system memory? We need to move the data over the PCIe bus (from the host to the GPU) using `cp.asarray()`. 

Modify the following cell to initialize the ndarray data with Numpy. 

How does the speedup change after the additional cost of data movement?

In [ ]:
size = int(1e8)

for i in range(3):
    print("Iteration ", i)
    start_time = perf_counter( )
    A_cpu=np.random.rand(size).astype(np.float32)
    B_cpu=np.random.rand(size).astype(np.float32)

    start_time = perf_counter( )
#>>>Insert CuPy code here
    stop_time = perf_counter( )
    gpu_time = stop_time - start_time
    
    print('cupy = %g seconds' % gpu_time )
    print("Speedup = %.2f" % (cpu_time/gpu_time))
    print('')

Click the `...` below to reveal the solution.

In [ ]:
size = int(1e8)

for i in range(3):
    print("Iteration ", i)
    
    start_time = perf_counter( )
    
    A_cpu=np.random.rand(size).astype(np.float32)
    B_cpu=np.random.rand(size).astype(np.float32)
    
    A_gpu=cp.asarray(A_cpu)
    B_gpu=cp.asarray(B_cpu)
    C_gpu = cp.dot(A_gpu,B_gpu)
    cp.cuda.Device(0).synchronize()
    
    stop_time = perf_counter( )
    gpu_time = stop_time - start_time
    
    print('cupy = %g seconds' % gpu_time )
    print("Speedup = %.2f" % (cpu_time/gpu_time))
    print('')


---

**Managing GPU Memory**

Modern datacenter GPUs have as much as 80GB of high-bandwidth memory on a single accelerator. But in general, the host system memory will have a larger capacity. We need to be conscious of GPU memory limitations when transfering data from the host. We can query the amount of free and total memory with nvidia-smi:

In [ ]:
!nvidia-smi -i 0 --query-gpu=memory.free,memory.total --format=csv

Or natively with CuPy

In [ ]:
print("GPU (free, total) memory in bytes:")
print(cp.cuda.Device().mem_info)

Let's clear all GPU memory for good measure. 

In [ ]:
cp.get_default_memory_pool().free_all_blocks()

print("GPU (free, total) memory in bytes:")
print(cp.cuda.Device().mem_info)

What happens if we try to allocate too much space on the GPU? In the following example, arrays A and B are 8GB each.

In [ ]:
size = 32768
A = cp.ones((size, size))
B = cp.ones((size, size)) 

One possible solution is to switch over to unified memory. With unified memory, the CUDA runtime will migrate data between the CPU and GPU _on demand_. Data migrations are triggered by page faults, so we may be leaving some performance on the table by using unified memory instead of managing memory explicitly. But it's an extremely convenient feature for making GPUs easier to program. We can enable Unified Memory in CuPy as follows:

In [ ]:
#Create a memory pool instance with malloc_managed allocator
pool = cp.cuda.MemoryPool(cp.cuda.malloc_managed)
cp.cuda.set_allocator(pool.malloc)

Let's try that again.

In [ ]:
size = 32768
A = cp.ones((size, size))
B = cp.ones((size, size))

We can certainly perform computations on these new arrays. Performance will take a hit as the GPU swaps pages in-and-out of memory

In [ ]:
cp.add(A,B)

---

**Please restart the kernel**

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)